In [1]:
#데이터 가져오기
import pickle 

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

In [2]:
#데이터 10개 추출
questions = rag_data['questions'][:10]
contexts = rag_data['contexts'][:10]
answers = rag_data['answers'][:10]
len(questions)

10

In [3]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

c:\Users\HyeonSeong\Hateslop\RAG-1stWeek\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 평가

### Baseline (w/o RAG)

In [18]:
#GPT 답변 추출
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
for i in tqdm(range(len(questions))):
    prompt = f"""Please answer to the question given below

Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [00:48<00:00,  4.84s/it]


In [19]:
predictions

["연구자들은 AI가 저작권이 있는 자료를 잊어버리도록 돕기 위해 여러 가지 접근 방식을 사용하고 있습니다. 그 중 몇 가지 주요 접근 방식은 다음과 같습니다:\n\n1. **데이터 필터링 및 정제**: AI 모델을 훈련시키기 전에 저작권이 있는 자료를 필터링하고 제거하는 방법입니다. 이를 통해 AI가 저작권이 있는 자료를 학습하지 않도록 합니다.\n\n2. **모델 수정 및 업데이트**: 이미 훈련된 AI 모델에서 저작권이 있는 자료를 제거하기 위해 모델을 수정하거나 업데이트하는 방법입니다. 이를 위해 특정 데이터를 모델에서 '잊어버리게' 하는 기술이 연구되고 있습니다.\n\n3. **데이터 마스킹**: 저작권이 있는 자료를 마스킹하거나 변형하여 AI가 원본 자료를 인식하지 못하게 하는 방법입니다. 이를 통해 AI가 저작권 침해 없이 학습할 수 있도록 합니다.\n\n4. **법적 및 윤리적 가이드라인 준수**: AI 연구자들은 저작권법과 윤리적 가이드라인을 준수하여 AI 모델을 개발하고 있습니다. 이를 통해 저작권 침해를 방지하고, AI가 합법적으로 자료를 사용할 수 있도록 합니다.\n\n5. **기술적 솔루션 개발**: AI가 특정 데이터를 잊어버리도록 하는 기술적 솔루션을 개발하는 연구도 진행 중입니다. 예를 들어, '모델 잊기(forgetting)' 기술을 통해 AI가 특정 데이터를 기억하지 않도록 하는 방법이 있습니다.\n\n이러한 접근 방식을 통해 연구자들은 AI가 저작권이 있는 자료를 잊어버리고, 저작권 침해 없이 학습할 수 있도록 노력하고 있습니다.",
 '2023년에 설립된 G7 AI 프레임워크의 목적은 인공지능(AI) 기술의 개발과 사용에 있어서 윤리적이고 책임 있는 접근을 촉진하는 것입니다. 이 프레임워크는 G7 국가들이 AI 기술의 잠재력을 최대한 활용하면서도, 개인의 프라이버시 보호, 데이터 보안, 공정성, 투명성 등의 중요한 윤리적 문제를 해결하기 위해 협력하는 것을 목표로 합니다. 이를 통해 AI 기술이 사회에 긍정적인 영향을 미치고

In [24]:
#FactualCorrectness: 생성된 답변이 주어진 문맥(또는 참조 데이터)과 비교했을 때 사실적으로 정확한지 평가
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

samples = []
for context, question, ai_response, ground_truth in zip(contexts, questions, predictions, answers):
    sample = SingleTurnSample(
        retrieve_contexts=[context],
        user_input=question,
        response=ai_response,
        reference=ground_truth
    )
    samples.append(sample)


dataset = EvaluationDataset(samples)

factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)

#준비된 데이터셋(dataset)과 메트릭(metrics)을 입력으로 받아, 평가 점수를 계산
results = evaluate(dataset, metrics=[factual_correctness_metric])

#평가 결과(results)를 Pandas 데이터프레임으로 변환 후 상위 5개 행 확인
results_df = results.to_pandas()
print(results_df.head())

Evaluating: 100%|██████████| 10/10 [00:41<00:00,  4.10s/it]


                                          user_input  \
0  연구자들은 AI가 저작권이 있는 자료를 잊어버리도록 돕기 위해 어떤 접근 방식을 사...   
1                 2023년에 설립된 G7 AI 프레임워크의 목적은 무엇인가요?   
2       현재 재단 모델과 관련하여 EU AI법 협상에 영향을 미치는 과제는 무엇인가요?   
3                  인공지능 규제와 관련하여 EU AI 법의 의미는 무엇인가요?   
4                 AI 애플리케이션의 맥락에서 성능 평가의 중요성은 무엇인가요?   

                                            response  \
0  연구자들은 AI가 저작권이 있는 자료를 잊어버리도록 돕기 위해 여러 가지 접근 방식...   
1  2023년에 설립된 G7 AI 프레임워크의 목적은 인공지능(AI) 기술의 개발과 사...   
2  현재 재단 모델과 관련하여 EU AI법 협상에 영향을 미치는 주요 과제는 다음과 같...   
3  EU AI 법, 공식적으로 "인공지능 법안" (Artificial Intellige...   
4  AI 애플리케이션의 맥락에서 성능 평가의 중요성은 매우 큽니다. 다음은 그 이유를 ...   

                                           reference  factual_correctness  
0  연구자들은 해리 포터를 통해 인공지능이 저작권이 있는 자료를 잊어버릴 수 있도록 돕...                 0.38  
1  2023년에 설립된 G7 AI 프레임워크의 목적은 AI 기술의 책임 있는 개발과 사...                 0.57  
2  프랑스, 독일, 이탈리아가 추진하고 있는 재단 모델에 대한 의무적 자율 규제의 필요...                 0.09  
3  EU 인공지능법은 인공지능 기술과 

In [25]:
print(results)

{'factual_correctness': 0.2830}


In [26]:
print(questions[0])

연구자들은 AI가 저작권이 있는 자료를 잊어버리도록 돕기 위해 어떤 접근 방식을 사용하고 있나요?


In [27]:
print(predictions[0])

연구자들은 AI가 저작권이 있는 자료를 잊어버리도록 돕기 위해 여러 가지 접근 방식을 사용하고 있습니다. 그 중 몇 가지 주요 접근 방식은 다음과 같습니다:

1. **데이터 필터링 및 정제**: AI 모델을 훈련시키기 전에 저작권이 있는 자료를 필터링하고 제거하는 방법입니다. 이를 통해 AI가 저작권이 있는 자료를 학습하지 않도록 합니다.

2. **모델 수정 및 업데이트**: 이미 훈련된 AI 모델에서 저작권이 있는 자료를 제거하기 위해 모델을 수정하거나 업데이트하는 방법입니다. 이를 위해 특정 데이터를 모델에서 '잊어버리게' 하는 기술이 연구되고 있습니다.

3. **데이터 마스킹**: 저작권이 있는 자료를 마스킹하거나 변형하여 AI가 원본 자료를 인식하지 못하게 하는 방법입니다. 이를 통해 AI가 저작권 침해 없이 학습할 수 있도록 합니다.

4. **법적 및 윤리적 가이드라인 준수**: AI 연구자들은 저작권법과 윤리적 가이드라인을 준수하여 AI 모델을 개발하고 있습니다. 이를 통해 저작권 침해를 방지하고, AI가 합법적으로 자료를 사용할 수 있도록 합니다.

5. **기술적 솔루션 개발**: AI가 특정 데이터를 잊어버리도록 하는 기술적 솔루션을 개발하는 연구도 진행 중입니다. 예를 들어, '모델 잊기(forgetting)' 기술을 통해 AI가 특정 데이터를 기억하지 않도록 하는 방법이 있습니다.

이러한 접근 방식을 통해 연구자들은 AI가 저작권이 있는 자료를 잊어버리고, 저작권 침해 없이 학습할 수 있도록 노력하고 있습니다.


In [ ]:
#context가 주어진 GPT 답변 추출
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
for i in tqdm(range(len(questions))):
    prompt = f"""Please answer to the question given below from the context

Context:
{contexts[i]}

Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [00:35<00:00,  3.57s/it]


In [29]:
#context가 주어진 상태의 답변 평가
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

samples = []
for context, question, ai_response, ground_truth in zip(contexts, questions, predictions, answers):
    sample = SingleTurnSample(
        retrieve_contexts=[context],
        user_input=question,
        response=ai_response,
        reference=ground_truth
    )
    samples.append(sample)


dataset = EvaluationDataset(samples)

factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)

#준비된 데이터셋(dataset)과 메트릭(metrics)을 입력으로 받아, 평가 점수를 계산
results = evaluate(dataset, metrics=[factual_correctness_metric])

#평가 결과(results)를 Pandas 데이터프레임으로 변환 후 상위 5개 행 확인
results_df = results.to_pandas()
print(results_df.head())

Evaluating: 100%|██████████| 10/10 [00:20<00:00,  2.09s/it]


                                          user_input  \
0  연구자들은 AI가 저작권이 있는 자료를 잊어버리도록 돕기 위해 어떤 접근 방식을 사...   
1                 2023년에 설립된 G7 AI 프레임워크의 목적은 무엇인가요?   
2       현재 재단 모델과 관련하여 EU AI법 협상에 영향을 미치는 과제는 무엇인가요?   
3                  인공지능 규제와 관련하여 EU AI 법의 의미는 무엇인가요?   
4                 AI 애플리케이션의 맥락에서 성능 평가의 중요성은 무엇인가요?   

                                            response  \
0  주어진 문맥에서는 연구자들이 AI가 저작권이 있는 자료를 잊어버리도록 돕기 위한 특...   
1  2023년에 설립된 G7 AI 프레임워크의 목적은 첨단 AI 시스템을 개발하는 기업...   
2  현재 재단 모델과 관련하여 EU AI법 협상에 영향을 미치는 과제는 다음과 같습니다...   
3  EU AI 법(AI Act)은 유럽연합(EU) 내에서 인공지능(AI) 기술의 개발과...   
4  AI 애플리케이션의 맥락에서 성능 평가의 중요성은 다음과 같은 이유들로 설명될 수 ...   

                                           reference  factual_correctness  
0  연구자들은 해리 포터를 통해 인공지능이 저작권이 있는 자료를 잊어버릴 수 있도록 돕...                 0.33  
1  2023년에 설립된 G7 AI 프레임워크의 목적은 AI 기술의 책임 있는 개발과 사...                 0.29  
2  프랑스, 독일, 이탈리아가 추진하고 있는 재단 모델에 대한 의무적 자율 규제의 필요...                 0.29  
3  EU 인공지능법은 인공지능 기술과 

In [30]:
print(results)

{'factual_correctness': 0.2160}


In [31]:
print(predictions[0])

주어진 문맥에서는 연구자들이 AI가 저작권이 있는 자료를 잊어버리도록 돕기 위한 특정 접근 방식에 대해 언급하고 있지 않습니다. 문맥은 주로 편향된 훈련 데이터에 대한 분석 기술, 민감한 정보를 포함하는 AI 시스템에 대한 미세 조정 방법, 그리고 정보 공유 채널 구축에 관한 내용을 다루고 있습니다.


In [32]:
print(answers[0])

연구자들은 해리 포터를 통해 인공지능이 저작권이 있는 자료를 잊어버릴 수 있도록 돕고 있습니다.
